# Natural Language Processing on Amazon Fine Food Reviews
## Assignment 2

**Aadith Sukumar**<br>
**AIML A1**<br>
**21070126003**<br>

Data Source: [Kaggle](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews?resource=download)

Questions:
1. Use the same dataset from Assignment 1.

2. DO FOLLOWING Preprocessing STEPS using NLTK or SpaCy:
    1. Remove empty rows. Remove duplicates.
    2. Tokenization, Lemmatization
    3. Data Cleansing: Remove stopwords, remove symbols,remove emojis,     remove URLS, remove http tags, remove excess whitespaces.
    4. Lower the strings, replace abbreviations, fix contractions.

3. Use following as range of values for input variables:
    1. For 1st set of results use:
      1. batch_size = 4
      2. max_sequence_length = 50
      2. embedding_dim = 50
      2. max_words = 10000
      2. lstm_units = 32
     
    2. For 2st set of results use:
      1. batch_size = 8
      1. max_sequence_length = 30
      1. embedding_dim = 30
      1. max_words = 25000
      1. lstm_units = 32

4. Use following LSTM:
    1. Single Layer of LSTM for 1st set of results
    2. Two Layers of LSTM for 2nd set of results

5. Compare the results using Classification Report.

## Importing Libraries

In [2]:
import pandas as pd
import nltk
nltk.download("all")
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

## 1. Loading The Dataset

In [3]:
data = pd.read_csv("/content/drive/MyDrive/NLP Datasets/Reviews.csv")
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
print("Shape of data: ",data.shape)
data.info()

Shape of data:  (568454, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


### Sampling Data for faster processing

In [ ]:
# sampling data to get 5000 rows
data = data.sample(n=5000)
data.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
256878,256879,B000QSN21K,A1O6G22V5906NT,"E. L. ""Artgirl""",0,0,5,1321833600,"My cats LOVE it, I have to reorder every 2-3 m...",I was looking for better quality wet food for ...
132736,132737,B006S8DD4I,A3T4X0SJQHV6V5,david,0,0,5,1348358400,Great Food!!,This is one of the BEST foods you can get for ...
33707,33708,B000CONMBS,A2Z6E2JHMTJJR4,Heez,0,0,4,1327017600,Delicious!,I first tried this at Walmart and I believe th...
531781,531782,B005K4Q34S,A1FK6GLHTRGT54,Scott,0,0,4,1334275200,Great Deal,I really like this product and at such a low p...
546666,546667,B004M8J9Y4,AZNK3DB49PJ9P,"Jason Kaechler ""TechnoPhile""",1,1,5,1316908800,Good Coffee,This is Good Coffee. One of the best t-discs ...
269335,269336,B005ISZAI6,A2F4E5CNCKF7OM,"Blade ""Blade""",3,3,4,1320192000,Absente now an Absinthe,Absente Absinthe Refined does now (since 2009)...
228763,228764,B0058AMY10,A3ARC3UIY3Z1KQ,"Patricia A. Cioffi ""SHOPPING JUNKIE""",2,6,1,1238976000,VERY DISAPPOINTED,I ORDERED KETTLE CHIPS IN THE FOLLOWING FLAVVO...
54410,54411,B0047IRA1I,A3FY80ONMKWORT,Jinny J. Baldridge,1,1,1,1327708800,looked like candy sat on shelf for a while,candy arrived in broken pieces. And package lo...
74321,74322,B0009F3PQ2,ABH4JYTO2902H,Katie Jandro,0,0,5,1295136000,Better than cough drops,GREAT. Best thing you can drink if you have a ...
499643,499644,B000CQID6U,A3QHL5OK16I6JK,Macaroni,1,1,5,1216080000,Favorite Tea,"Stash Earl Grey Tea is my absolute favorite, b..."


In [ ]:
print("Shape of data: ",data.shape)
data.info()

Shape of data:  (5000, 10)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 256878 to 59211
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      5000 non-null   int64 
 1   ProductId               5000 non-null   object
 2   UserId                  5000 non-null   object
 3   ProfileName             4999 non-null   object
 4   HelpfulnessNumerator    5000 non-null   int64 
 5   HelpfulnessDenominator  5000 non-null   int64 
 6   Score                   5000 non-null   int64 
 7   Time                    5000 non-null   int64 
 8   Summary                 5000 non-null   object
 9   Text                    5000 non-null   object
dtypes: int64(5), object(5)
memory usage: 429.7+ KB


## 2. Preprocessing Data

### a. Lemmatization with Spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')
data['Text'] = data['Text'].apply(lambda x: nlp(x))

### a. Tokenization

In [ ]:
# Tokenization
data['Text'] = data['Text'].apply(lambda x: [item.text for item in x])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
256878,256879,B000QSN21K,A1O6G22V5906NT,"E. L. ""Artgirl""",0,0,5,1321833600,"My cats LOVE it, I have to reorder every 2-3 m...","[I, was, looking, for, better, quality, wet, f..."
132736,132737,B006S8DD4I,A3T4X0SJQHV6V5,david,0,0,5,1348358400,Great Food!!,"[This, is, one, of, the, BEST, foods, you, can..."


### b. Stop Words

In [ ]:
# removing stop words
data['Text'] = data['Text'].apply(lambda x: [item for item in x if item not in STOP_WORDS])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
256878,256879,B000QSN21K,A1O6G22V5906NT,"E. L. ""Artgirl""",0,0,5,1321833600,"My cats LOVE it, I have to reorder every 2-3 m...","[I, looking, better, quality, wet, food, cats,..."
132736,132737,B006S8DD4I,A3T4X0SJQHV6V5,david,0,0,5,1348358400,Great Food!!,"[This, BEST, foods, dogs, !, !, cost, plus, ea..."


### b. Symbols and Special Characters

In [ ]:
# remove symbols and special characters
data['Text'] = data['Text'].apply(lambda x: [item for item in x if item.isalpha()])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
256878,256879,B000QSN21K,A1O6G22V5906NT,"E. L. ""Artgirl""",0,0,5,1321833600,"My cats LOVE it, I have to reorder every 2-3 m...","[I, looking, better, quality, wet, food, cats,..."
132736,132737,B006S8DD4I,A3T4X0SJQHV6V5,david,0,0,5,1348358400,Great Food!!,"[This, BEST, foods, dogs, cost, plus, eat, I, ..."


### b. Removing URLs

In [ ]:
# urls
data['Text'] = data['Text'].apply(lambda x: [item for item in x if "http" not in item])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
256878,256879,B000QSN21K,A1O6G22V5906NT,"E. L. ""Artgirl""",0,0,5,1321833600,"My cats LOVE it, I have to reorder every 2-3 m...","[I, looking, better, quality, wet, food, cats,..."
132736,132737,B006S8DD4I,A3T4X0SJQHV6V5,david,0,0,5,1348358400,Great Food!!,"[This, BEST, foods, dogs, cost, plus, eat, I, ..."


---

## 3. Vectorizing Data

### CountVectorizer

In [ ]:
# Vectorizing using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

data['Text_Processed'] = data['Text'].apply(lambda x: ' '.join(x))
cv = CountVectorizer(max_features=5000, stop_words='english')
X_CV = cv.fit_transform(data['Text_Processed']).toarray()
print("X_CV-Shape: ",X_CV.shape)

X_CV-Shape:  (5000, 5000)


### TFIDFVectorizer

In [ ]:
# Vectorizing using TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X_TF = tfidf.fit_transform(data['Text_Processed']).toarray()
print("X_TF-Shape: ",X_TF.shape)

X_TF-Shape:  (5000, 5000)


### Word2Vec

In [ ]:
# Vectorizing using Word2Vec
from gensim.models import Word2Vec

model = Word2Vec(data['Text'], min_count=1)
print("model-Shape: ",model.wv.vectors.shape)

model-Shape:  (17465, 100)


### Loading GoogleNews Word2Vec

In [ ]:
# Vectorizing using GoogleNews Word2Vec
from gensim.models import KeyedVectors

model_GN = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary=True)
print("model_GN-Shape: ",model_GN.vectors.shape)

model_GN-Shape:  (3000000, 300)


---

## 4. ML Algorithms

### Loading Libraries

In [ ]:
# Logistic Regression with CountVectorizer, TFIDFVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

### Train Test Split

In [ ]:
# train test split for count vectorizer
X_train_CV, X_test_CV, y_train_CV, y_test_CV = train_test_split(X_CV, data['Score'], test_size=0.2, random_state=0)

# train test split for tfidf vectorizer
X_train_TF, X_test_TF, y_train_TF, y_test_TF = train_test_split(X_TF, data['Score'], test_size=0.2, random_state=0)

In [ ]:
# Logistic Regression with CountVectorizer
model_CV_LR = LogisticRegression()
model_CV_LR.fit(X_train_CV, y_train_CV)
y_pred_CV_LR = model_CV_LR.predict(X_test_CV)
print("Logistic Regression Accuracy Score with CountVectorizer: ",accuracy_score(y_test_CV, y_pred_CV_LR))

Logistic Regression Accuracy Score with CountVectorizer:  0.662


### Model Building

In [ ]:
# Logistic Regression with TFIDFVectorizer
model_TF_LR = LogisticRegression()
model_TF_LR.fit(X_train_TF, y_train_TF)
y_pred_TF_LR = model_TF_LR.predict(X_test_TF)
print("Logistic Regression Accuracy Score with TFIDFVectorizer: ",accuracy_score(y_test_TF, y_pred_TF_LR))

Logistic Regression Accuracy Score with TFIDFVectorizer:  0.661


In [ ]:
# SVC for CountVectorizer
model_CV_SVC = SVC(kernel='linear', C=1, random_state=0)
model_CV_SVC.fit(X_train_CV, y_train_CV)
y_pred_CV_SVC = model_CV_SVC.predict(X_test_CV)
print("SVC Accuracy Score with CountVectorizer: ",accuracy_score(y_test_CV, y_pred_CV_SVC))

SVC Accuracy Score with CountVectorizer:  0.617


In [ ]:
# SVC for TFIDFVectorizer
model_TF_SVC = SVC()
model_TF_SVC.fit(X_train_TF, y_train_TF)
y_pred_TF_SVC = model_TF_SVC.predict(X_test_TF)
print("SVC Accuracy Score with TFIDFVectorizer: ",accuracy_score(y_test_TF, y_pred_TF_SVC))

SVC Accuracy Score with TFIDFVectorizer:  0.658


In [ ]:
# Random Forest for CountVectorizer
random_forest_CV = RandomForestClassifier(n_estimators=100)
random_forest_CV.fit(X_train_CV, y_train_CV)
y_pred_CV_RF = random_forest_CV.predict(X_test_CV)
print("Random Forest Accuracy Score with CountVectorizer: ",accuracy_score(y_test_CV, y_pred_CV_RF))

Random Forest Accuracy Score with CountVectorizer:  0.669


In [ ]:
# Random Forest for TFIDFVectorizer
random_forest_TF = RandomForestClassifier(n_estimators=100)
random_forest_TF.fit(X_train_TF, y_train_TF)
y_pred_TF_RF = random_forest_TF.predict(X_test_TF)
print("Random Forest Accuracy Score with TFIDFVectorizer: ",accuracy_score(y_test_TF, y_pred_TF_RF))

Random Forest Accuracy Score with TFIDFVectorizer:  0.665


---

## 5. Comparison

In [ ]:
# Comparing all the models using Classification Report
from sklearn.metrics import classification_report

print("Logistic Regression with CountVectorizer")
print(classification_report(y_test_CV, y_pred_CV_LR))

print("Logistic Regression with TFIDFVectorizer")
print(classification_report(y_test_TF, y_pred_TF_LR))

print("SVC with CountVectorizer")
print(classification_report(y_test_CV, y_pred_CV_SVC))

print("SVC with TFIDFVectorizer")
print(classification_report(y_test_TF, y_pred_TF_SVC))

print("Random Forest with CountVectorizer")
print(classification_report(y_test_CV, y_pred_CV_RF))

print("Random Forest with TFIDFVectorizer")
print(classification_report(y_test_TF, y_pred_TF_RF))

Logistic Regression with CountVectorizer
              precision    recall  f1-score   support

           1       0.60      0.49      0.54       105
           2       0.16      0.11      0.13        44
           3       0.20      0.08      0.12        72
           4       0.28      0.20      0.23       135
           5       0.76      0.89      0.82       644

    accuracy                           0.66      1000
   macro avg       0.40      0.35      0.37      1000
weighted avg       0.61      0.66      0.63      1000

Logistic Regression with TFIDFVectorizer
              precision    recall  f1-score   support

           1       0.73      0.18      0.29       105
           2       0.00      0.00      0.00        44
           3       0.67      0.03      0.05        72
           4       0.21      0.05      0.08       135
           5       0.68      0.98      0.80       644

    accuracy                           0.66      1000
   macro avg       0.46      0.25      0.25      

---
# Thank You!